# Recommendation Pipeline


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import ast
from typing import List, Dict, Union, Tuple, Optional
from geopy.geocoders import Nominatim
import annoy
from pathlib import Path
import time
import networkx as nx
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from geopy.distance import geodesic
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import pickle
import faiss
from dataclasses import dataclass
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
# import ollama

# Initialization of job recommendation system components

# Define cache paths
CACHE_DIR = Path('recommendation_cache')
CACHE_DIR.mkdir(exist_ok=True)

CACHE_PATHS = {
    'graph': CACHE_DIR / 'final_complete_graph.pkl',
    'annoy_index': CACHE_DIR / 'annoy_index.ann',
    'faiss_index': CACHE_DIR / 'faiss_index.pkl', 
    'graph_metrics': CACHE_DIR / 'graph_metrics.pkl',
    'embeddings_norm': CACHE_DIR / 'normalized_embeddings.pkl',
    'degree_scores': CACHE_DIR / 'degree_scores.pkl',
    'subgraphs': CACHE_DIR / 'subgraphs.pkl',
    'model': CACHE_DIR / 'model_cache.pkl',
    'node_embeddings': CACHE_DIR / 'node_embeddings.pt'
}

# Load node embeddings
print("Loading node embeddings...")
if CACHE_PATHS['node_embeddings'].exists():
    print("Loading node embeddings from cache...")
    node_embeddings = torch.load(CACHE_PATHS['node_embeddings'])
    print("Node embeddings loaded successfully!")
else:
    raise FileNotFoundError("Node embeddings file not found. Please ensure node embeddings have been generated and saved from step 8.")

# MultiLabelBinarizer setup for job type decoding
mlb = MultiLabelBinarizer()
mlb.fit([['contract'], ['fulltime'], ['internship'], ['parttime'], ['temporary']])

# Geocoder initialization
geolocator = Nominatim(user_agent="job_recommender_v1")

# Load or initialize model and tokenizer
def load_or_init_model():
    print("Loading/initializing language model...")
    if CACHE_PATHS['model'].exists():
        print("Loading model from cache...")
        with open(CACHE_PATHS['model'], 'rb') as f:
            cache = pickle.load(f)
            tokenizer = cache['tokenizer']
            model = cache['model']
        print("Model loaded from cache successfully!")
    else:
        print("Downloading and caching model...")
        tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
        model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
        
        with open(CACHE_PATHS['model'], 'wb') as f:
            pickle.dump({
                'tokenizer': tokenizer,
                'model': model
            }, f)
        print("Model downloaded and cached successfully!")
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    model = model.to(device)
    return tokenizer, model, device

# Initialize tokenizer, model, and device
tokenizer, model, device = load_or_init_model()

# Load graph if available
if CACHE_PATHS['graph'].exists():
    with open(CACHE_PATHS['graph'], 'rb') as f:
        graph = pickle.load(f)
else:
    print("Error: Graph pickle file does not exist. Please check the path.")


In [4]:
import traceback
# Function to build Annoy index
def build_ann_index(embeddings_np, n_trees=100):
    print("\nBuilding/Loading Annoy index...")
    if CACHE_PATHS['annoy_index'].exists():
        print("Loading cached Annoy index...")
        index = annoy.AnnoyIndex(embeddings_np.shape[1], 'angular')
        index.load(str(CACHE_PATHS['annoy_index']))
        print("Annoy index loaded successfully!")
        return index
    
    print("Building new Annoy index...")
    index = annoy.AnnoyIndex(embeddings_np.shape[1], 'angular')
    
    for i in range(len(embeddings_np)):
        if i % 1000 == 0:
            print(f"Adding item {i}/{len(embeddings_np)} to Annoy index...")
        index.add_item(i, embeddings_np[i])
    
    print("Building index with trees...")
    index.build(n_trees)
    print("Saving index to disk...")
    index.save(str(CACHE_PATHS['annoy_index']))
    print("Annoy index built and saved successfully!")
    return index

# Function to build FAISS index
def build_faiss_index(embeddings):
    print("\nBuilding/Loading FAISS index...")
    if CACHE_PATHS['faiss_index'].exists():
        print("Loading cached FAISS index...")
        with open(CACHE_PATHS['faiss_index'], 'rb') as f:
            index = pickle.load(f)
        print("FAISS index loaded successfully!")
        return index
            
    print("Building new FAISS index...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    print("Normalizing embeddings...")
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1)[:, np.newaxis]
    print("Adding vectors to index...")
    index.add(normalized_embeddings.astype('float32'))
    
    print("Saving index to disk...")
    with open(CACHE_PATHS['faiss_index'], 'wb') as f:
        pickle.dump(index, f)
    print("FAISS index built and saved successfully!")
    return index

# Function to cache normalized embeddings
def cache_normalized_embeddings(embeddings_np):
    print("\nPreparing normalized embeddings...")
    if CACHE_PATHS['embeddings_norm'].exists():
        print("Loading cached normalized embeddings...")
        with open(CACHE_PATHS['embeddings_norm'], 'rb') as f:
            normalized = pickle.load(f)
        print("Normalized embeddings loaded successfully!")
        return normalized
    
    print("Computing normalized embeddings...")
    normalized = embeddings_np / np.linalg.norm(embeddings_np, axis=1)[:, np.newaxis]
    print("Saving normalized embeddings to disk...")
    with open(CACHE_PATHS['embeddings_norm'], 'wb') as f:
        pickle.dump(normalized, f)
    print("Normalized embeddings cached successfully!")
    return normalized

def compute_pagerank_torch(graph, damping=0.85, max_iter=100, tol=1e-6):
    print("\nComputing PageRank scores...")
    # Create node ID to index mapping
    node_map = {node: idx for idx, node in enumerate(graph.nodes())}
    reverse_map = {idx: node for node, idx in node_map.items()}
    n = len(node_map)
    
    # Convert edges using the mapping
    edges = [(node_map[e[0]], node_map[e[1]]) for e in graph.edges()]
    row = np.array([e[0] for e in edges])
    col = np.array([e[1] for e in edges])
    data = np.ones(len(edges))
    adj_matrix = csr_matrix((data, (row, col)), shape=(n, n))

    print("Normalizing adjacency matrix...")
    out_degree = np.array(adj_matrix.sum(axis=1)).flatten()
    out_degree[out_degree == 0] = 1
    D_inv = csr_matrix((1.0 / out_degree, (np.arange(n), np.arange(n))), shape=(n, n))
    stochastic_matrix = D_inv @ adj_matrix

    print("Converting to PyTorch sparse format...")
    coo_matrix = stochastic_matrix.tocoo()
    indices = torch.tensor([coo_matrix.row, coo_matrix.col], dtype=torch.long)
    values = torch.tensor(coo_matrix.data, dtype=torch.float32)
    sparse_matrix = torch.sparse.FloatTensor(indices, values, torch.Size([n, n])).cuda()

    print("Initializing PageRank computation...")
    pagerank_vector = torch.ones(n, device="cuda") / n

    for i in range(max_iter):
        new_pagerank_vector = (1 - damping) / n + damping * torch.sparse.mm(sparse_matrix.t(), pagerank_vector.unsqueeze(1)).squeeze()
        if torch.norm(new_pagerank_vector - pagerank_vector, p=1) < tol:
            print(f"PageRank converged after {i + 1} iterations")
            break
        pagerank_vector = new_pagerank_vector

    print("PageRank computation completed!")
    # Convert back to original node IDs
    pagerank_dict = {reverse_map[i]: score for i, score in enumerate(pagerank_vector.cpu().numpy())}
    return pagerank_dict

# Function to cache graph metrics
def cache_graph_metrics(graph):
    print("\nComputing/Loading graph metrics...")
    if CACHE_PATHS['graph_metrics'].exists():
        print("Loading cached graph metrics...")
        with open(CACHE_PATHS['graph_metrics'], 'rb') as f:
            metrics = pickle.load(f)
        print("Graph metrics loaded successfully!")
        return metrics
    
    print("Computing graph metrics...")
    print("Computing degree centrality...")
    degree = dict(graph.degree())
    max_degree = max(degree.values())
    
    print("Computing PageRank...")
    pagerank = compute_pagerank_torch(graph)
    
    print("Computing core numbers...")
    core_numbers = nx.core_number(graph)
    
    metrics = {
        'degree': degree,
        'max_degree': max_degree,
        'pagerank': pagerank,
        'core_numbers': core_numbers
    }
    
    print("Saving graph metrics to disk...")
    with open(CACHE_PATHS['graph_metrics'], 'wb') as f:
        pickle.dump(metrics, f)
    print("Graph metrics cached successfully!")
    return metrics

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def get_job_description_embedding(text, tokenizer, model, max_chunk_length=512, overlap=50, device='cpu'):
    print("\nGenerating job description embedding...")
    tokens_per_chunk = max_chunk_length - 2  
    words = text.split()
    chunks = [' '.join(words[i:i+tokens_per_chunk]) for i in range(0, len(words), tokens_per_chunk - overlap)]
    
    embeddings = []
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}...")
        encoded_input = tokenizer(chunk, return_tensors='pt', padding=True, truncation=True, max_length=max_chunk_length).to(device)
        
        with torch.no_grad():
            model_output = model(**encoded_input)
        
        sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
        sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)
        embeddings.append(sentence_embedding.cpu().numpy())

    if embeddings:
        print("Averaging chunk embeddings...")
        return np.mean(embeddings, axis=0).squeeze()
    else:
        print("Warning: No embeddings generated, returning zero vector")
        return np.zeros((384,))

def get_job_title_embedding(text, tokenizer, model, device='cpu'):
    print(f"\nGenerating embedding for job title: {text}")
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(device)
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)
    print("Job title embedding generated successfully!")
    
    return sentence_embedding.cpu().numpy().squeeze()

# def process_job_description_with_LLM(document_text):
#     """Process a single document using the same LLM setup."""
#     model_name = 'capybarahermes-2.5-mistral-7b.Q5_K_M.gguf:latest'
#     prompt = f"""You are an expert in understanding job descriptions and extracting the details and even nuanced requirements for the job. Your goal is to read the input slowly and take time to consider what is written, extract the information and break it down into these 3 aspects:
#     1. responsibilites 
#     2. qualifications
#     3. skills, technical and non-technical
# and summarize it in point form line by line.
# With each aspect answered, ensure that each of the aspects are properly differentiated and avoid overlaps as much as possible."""
    
#     try:
#         response = ollama.chat(
#             model=model_name,
#             messages=[
#                 {'role': 'system', 'content': prompt},
#                 {'role': 'user', 'content': document_text}
#             ]
#         )
#         response_text = response['message']['content']
        
#         # Clean the response text
#         # Remove special characters except alphanumeric, spaces, periods and commas
#         cleaned_text = re.sub(r'[^A-Za-z0-9\s.,]', '', response_text)
#         # Remove point formatted numbers but keep time patterns
#         cleaned_text = re.sub(r'(?<!\d)(\d+)\.(?!\d)', '', cleaned_text).strip()
        
#         return cleaned_text
#     except Exception as e:
#         print(f"Error processing document: {e}")
#         return None

@dataclass
class UserPreferences:
    """User preferences for job recommendations"""
    location: Optional[Tuple[float, float]] = None
    location_name: Optional[str] = None  # Added for geocoding
    job_title: Optional[str] = None
    job_description: Optional[str] = None
    max_distance_km: float = 10.0
    weights: dict = None
    remote_preference: bool = False
    
    def __post_init__(self):
        if self.weights is None:
            self.weights = {
                'title_similarity': 0.3,
                'description_similarity': 0.2,
                'location_proximity': 0.2,
                'degree': 0.1,
                'pagerank': 0.1,
                'core_number': 0.1
            }

def get_graph_based_recommendations(
    graph: nx.Graph,
    node_embeddings: torch.Tensor,
    preferences: UserPreferences,
    n_hops: int = 2,
    top_k: int = 5,
    n_candidates: int = 1000
) -> List[dict]:
    """Generate recommendations using both FAISS and Annoy for hybrid search"""
    print("\nStarting recommendation generation...")
    start_time = time.time()
    
    print("Loading job URL data...")
    df = pd.read_pickle('./recommendation_cache/final_complete_graph_dataframe.pkl')
    
    print("Preparing embeddings...")
    embeddings_np = node_embeddings.numpy()
    print(f"Embeddings shape: {embeddings_np.shape}")
    
    print("\nInitializing search indices...")
    faiss_index = build_faiss_index(embeddings_np)
    annoy_index = build_ann_index(embeddings_np)
    normalized_embeddings = cache_normalized_embeddings(embeddings_np)
    graph_metrics = cache_graph_metrics(graph)
    
    print("\nGenerating candidate nodes...")
    candidate_nodes = set()
    if preferences.job_title:
        print(f"Finding similar jobs to title: {preferences.job_title}")
        title_embedding = get_job_title_embedding(preferences.job_title, tokenizer, model, device)
        title_embedding = title_embedding / np.linalg.norm(title_embedding)
        
        print("Searching with FAISS...")
        D_faiss, I_faiss = faiss_index.search(title_embedding.astype('float32').reshape(1,-1), top_k)
        faiss_candidates = {f"job_{idx}" for idx in I_faiss[0]}
        
        print("Searching with Annoy...")
        annoy_candidates = {f"job_{idx}" for idx in annoy_index.get_nns_by_vector(
            title_embedding, 
            top_k,
            search_k=-1
        )}
        
        candidate_nodes = faiss_candidates.union(annoy_candidates)
        print(f"Found {len(candidate_nodes)} candidate nodes")
    else:
        print("No job title specified, using PageRank for candidate selection...")
        weights = np.array([graph_metrics['pagerank'][node] for node in graph.nodes()])
        weights = weights / weights.sum()
        candidate_nodes = set(np.random.choice(
            list(graph.nodes()),
            size=min(n_candidates, len(graph)),
            p=weights,
            replace=False
        ))
    
    print("\nComputing job scores...")
    subgraph = graph.subgraph(candidate_nodes)
    degree_scores = dict(subgraph.degree())
    max_degree = max(degree_scores.values()) if degree_scores else 1
    
    batch_size = 100
    job_scores = []
    total_batches = len(candidate_nodes) // batch_size + (1 if len(candidate_nodes) % batch_size else 0)
    
    for i in range(0, len(candidate_nodes), batch_size):
        batch_nodes = list(candidate_nodes)[i:i + batch_size]
        current_batch = i // batch_size + 1
        print(f"\nProcessing batch {current_batch}/{total_batches}...")
        batch_scores = []
        
        for node in batch_nodes:
            try:
                node_id = node  # Keep the full node ID (e.g. 'job_0')
                attrs = graph.nodes[node]
                
                score_components = {}
                
                if preferences.job_title:
                    score_components['title_similarity'] = np.dot(
                        title_embedding,
                        normalized_embeddings[int(node.split('_')[1])]  # Extract number from 'job_X'
                    )
                if preferences.job_description:
                    desc_embedding = get_job_description_embedding(
                        preferences.job_description,
                        tokenizer,
                        model,
                        device=device
                    )
                    score_components['description_similarity'] = np.dot(
                        desc_embedding,
                        normalized_embeddings[int(node.split('_')[1])]  # Extract number from 'job_X'
                    )
                if preferences.location:
                    job_location = (
                        ast.literal_eval(attrs['lat_long'])
                        if isinstance(attrs['lat_long'], str)
                        else attrs['lat_long']
                    )
                    distance = geodesic(preferences.location, job_location).kilometers
                    score_components['location_proximity'] = max(0, 1 - (distance / preferences.max_distance_km))
                
                score_components['degree'] = degree_scores[node] / max_degree
                score_components['pagerank'] = graph_metrics['pagerank'][node]
                score_components['core_number'] = graph_metrics['core_numbers'][node] / max(graph_metrics['core_numbers'].values())
                
                final_score = sum(
                    score * preferences.weights.get(component, 0.1)
                    for component, score in score_components.items()
                )
                
                job_type_encoded = attrs['job_type_encoding']
                if isinstance(job_type_encoded, str):
                    job_type_encoded = ast.literal_eval(job_type_encoded)
                job_type_encoded = np.array(job_type_encoded)
                if job_type_encoded.ndim == 1:
                    job_type_encoded = job_type_encoded.reshape(1, -1)

                job_type_decoded = mlb.inverse_transform(job_type_encoded)[0]

                batch_scores.append({
                    'node_id': node,
                    'job_id': node,  # Keep the full node ID (e.g. 'job_0')
                    'company': attrs['company'],
                    'job_type': job_type_decoded,
                    'location': attrs['lat_long'],
                    'is_remote': attrs['is_remote'],
                    'score_components': score_components,
                    'final_score': final_score
                })
                
            except Exception as e:
                print(f"Error processing node {node}: {str(e)}")
                traceback.print_exc()
                continue
        
        job_scores.extend(batch_scores)
    
    print("\nSorting recommendations...")
    recommendations = sorted(
        job_scores,
        key=lambda x: x['final_score'],
        reverse=True
    )[:top_k]
    
    # Get URLs and titles for only the top recommendations
    for rec in recommendations:
        job_data = df[df['node_id'] == rec['job_id']].iloc[0]
        rec['job_url'] = job_data['job_url']
        rec['job_url_direct'] = job_data['job_url_direct']
        rec['title'] = job_data['title']  # Add job title to recommendations
    
    print(f"\nRecommendation generation completed in {time.time() - start_time:.2f} seconds")
    return recommendations

def get_personalized_recommendations(preload_preferences=False):
    """Interactive function to get user preferences and return recommendations"""
    print("\nJob Recommendation System")
    print("------------------------")
    
    preferences = UserPreferences()
    
    if preload_preferences:
        # Preload default preferences
        preferences.location = (1.3521, 103.8198)  # Singapore coordinates
        preferences.location_name = "Singapore"
        preferences.max_distance_km = 10
        preferences.job_title = "Service Technician"
        preferences.job_description = "Skilled in mechanical work, always hardworking, able to handle odd hours"
        preferences.weights = {
            'title_similarity': 0.3,
            'description_similarity': 0.2,
            'location_proximity': 0.2,
            'degree': 0.1,
            'pagerank': 0.1,
            'core_number': 0.1
        }
        print("Using preloaded preferences:")
        print(f"Location: {preferences.location_name}")
        print(f"Job Title: {preferences.job_title}")
        print(f"Job Description: {preferences.job_description}")
        print(f"Max Distance: {preferences.max_distance_km} km")
        print("Weights:", preferences.weights)
    else:
        use_location = input("Would you like to specify a location? (y/n): ").lower() == 'y'
        if use_location:
            location_input = int(input("Enter Postal Code (e.g., 123456): "))
            try:
                print("Geocoding location...")
                location = geolocator.geocode(location_input)
                if location:
                    preferences.location = (location.latitude, location.longitude)
                    preferences.location_name = location_input
                    print(f"Location found: {location.address}")
                else:
                    print("Location not found. Please enter coordinates manually.")
                    lat = float(input("Enter latitude (e.g., 1.3521 for Singapore): "))
                    lon = float(input("Enter longitude (e.g., 103.8198): "))
                    preferences.location = (lat, lon)
            except Exception as e:
                print(f"Error geocoding location: {str(e)}")
                print("Please enter coordinates manually.")
                lat = float(input("Enter latitude (e.g., 1.3521 for Singapore): "))
                lon = float(input("Enter longitude (e.g., 103.8198): "))
                preferences.location = (lat, lon)
            
            preferences.max_distance_km = float(input("Maximum distance in km (default 10): ") or 10)
        
        use_title = input("Would you like to specify a job title? (y/n): ").lower() == 'y'
        if use_title:
            preferences.job_title = input("Enter job title: ")
        
        use_desc = input("Would you like to specify a job description? (y/n): ").lower() == 'y'
        if use_desc:
            preferences.job_description = input("Enter job description: ")
        
        print("\nSet the importance for each of the following factors on a scale from 0 (not important) to 1 (very important):")
        if use_title:
            preferences.weights['title_similarity'] = float(input("How important is it for the job title to match your preferences? (default 0.3): ") or 0.3)
        if use_desc:
            preferences.weights['description_similarity'] = float(input("How important is it for the job description to match your skills and experience? (default 0.2): ") or 0.2)
        if use_location:
            preferences.weights['location_proximity'] = float(input("How important is it for the job to be near your preferred location? (default 0.2): ") or 0.2)

        preferences.weights['degree'] = float(input("How important is it for the job to be popular or well-connected within the network? (default 0.1): ") or 0.1)
        preferences.weights['pagerank'] = float(input("How important is it for the job to be influential within the network? (default 0.1): ") or 0.1)
        preferences.weights['core_number'] = float(input("How important is it for the job to be well-connected within its area or community? (default 0.1): ") or 0.1)
    
    print("\nGenerating recommendations...")
    recommendations = get_graph_based_recommendations(
        graph=graph,
        node_embeddings=node_embeddings,
        preferences=preferences
    )
    
    print("\nTop Recommendations:")
    print("-------------------")
    for i, rec in enumerate(recommendations, 1):
        print(f"\n{i}. Company: {rec['company']}")
        print(f"   Job Title: {rec['title']}")  # Added job title to output
        print(f"   Job Type: {', '.join(rec['job_type'])}")
        if preferences.location:
            job_loc = ast.literal_eval(rec['location']) if isinstance(rec['location'], str) else rec['location']
            distance = geodesic(preferences.location, job_loc).kilometers
            print(f"   Distance: {distance:.1f} km")
        print(f"   Remote: {rec['is_remote']}")
        print(f"   Job URL: {rec['job_url']}")
        print(f"   Direct URL: {rec['job_url_direct']}")
        print("   Scores:")
        for component, score in rec['score_components'].items():
            print(f"      - {component}: {score:.3f}")
        print(f"   Final Score: {rec['final_score']:.3f}")
